<a href="https://colab.research.google.com/github/danielmlow/tutorials/blob/main/plotting_and_io/python_to_r_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run R and Python in the same script (checked on Python 3.7.12 and 3.10.12)

In [ ]:
# !pip uninstall -y rpy2 # if it's installed
!pip install rpy2==3.5.1



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=314932 sha256=f1c00008eae17463910c4457f3a135c86c53ad6261527f093c68d45af75ecebb
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.4.2
    Uninstalling rpy2-3.4.2:
      Successfully uninstalled rpy2-3.4.2


In [ ]:

!python --version # Python 3.10.12

Python 3.10.12


In [2]:

import rpy2
rpy2.__version__

'3.4.2'

In [3]:
# activate %%R magic
%load_ext rpy2.ipython

# Import pandas
import pandas as pd
# Import rpy2 for dataframe conversion
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import globalenv

def p2r(df, df_name):

  '''
  Python to R dataframe conversion

  df: Pandas dataframe
  df_name: str, name of the dataframe
  '''
  # Convert the python dataframe to the R dataframe
  with localconverter(ro.default_converter + pandas2ri.converter):
    dfr = ro.conversion.py2rpy(df)
  # Create a variable name in R's global environment
  globalenv[df_name] = dfr
  return

If you get this error on Mac OS M1:


`OSError: cannot load library '/Library/Frameworks/R.framework/Resources/lib/libR.dylib': dlopen(/Library/Frameworks/R.framework/Resources/lib/libR.dylib, 0x0002)`


you might have to do this:
1. Install the x86 version of the R framework at https://cran.r-project.org/bin/macosx/base/R-4.2.3.pkg .
2. Open terminal Then change the link by running the command:

 `sudo ln -s /Library/Frameworks/R.framework/Versions/4.2 /Library/Frameworks/R.framework/Versions/Current`

In [4]:
%%R
a <- 1
a

[1] 1


### Share Python variables to R cells

In [5]:
list_1 = ['a', 'b', 'c']
%Rpush list_1

In [6]:
%%R
list_1


[[1]]
[1] "a"

[[2]]
[1] "b"

[[3]]
[1] "c"



### Create a dataframe in Python and save to R

In [ ]:
toy_df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
display(toy_df)
p2r(toy_df, 'toy_df')

,a,b
0,1,4
1,2,5
2,3,6


/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:54: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


### Use R

In [ ]:
%%R
# Do something in R
toy_df <- toy_df+1
print(class(toy_df))
print(toy_df)





[1] "data.frame"
  a b
0 2 5
1 3 6
2 4 7


### Use Python and access R DFs using `%Rget`

In [ ]:
toy_df # Before accessing R DF

,a,b
0,1,4
1,2,5
2,3,6


In [ ]:
# After accessing R DF
toy_df = %Rget toy_df
toy_df

,a,b
0,2.0,5.0
1,3.0,6.0
2,4.0,7.0


In [ ]:
%%R

install.packages('pacman', repos='http://cran.us.r-project.org') # you need to add this repos argument when using R locally from Python Jupyter notebook
install.packages('mgcv', repos='http://cran.us.r-project.org')
install.packages('lme4', repos='http://cran.us.r-project.org')